#COBRAme Reaction Buidling

In [1]:
import cobrame
from cobrame.util import dogma, building
import cobra
import cobra.test
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

## Overview

COBRAme is constructed entirely over COBRApy. This means that ME reactions will have all of the same properties, methods, and functions as a COBRApy reaction. However, one key difference is that, in order to facilliate the template nature of many gene expression reactions, reactions are constructed and their components are manipulated via the use of the ProcessData classes. These act as information vessels for holding the information assocatied with a cellular process in simple, standard datatypes such as dictionaries and strings. 

-----

This tutorial will go step-by-step through the process of creating the generic enzyme catalyzed reaction or metabolic reaction:
$$ a \rightarrow b $$
that requires the formation of **complex\_ab** in order to proceed

-------

In order for this reaction to carry flux in the model we will additionally need to add the corresponding:
1. Complex formation reaction
2. Translation reactions
3. tRNA charging reactions
4. Transcription reactions

Once these are added we will add in synthesis of key macromolecular components (ribosome, Degradosome, etc.) and show how they are applied to their respective reactions with their coupling constraints.

## Initializing new ME-models

When applying some constraints in the ME-model, metabolite properties are required. For instance, to calculate the total biomass produced by a particular reaction, the amino acid, nucleotide, etc. molecular weights are required.

To enable these calculations, all metabolites from iJO1366, along with their metabolite attributes are added to the newly created ME-model.

In [2]:
# create empty ME-model
me = cobrame.MEmodel('test')
ijo = cobra.test.create_test_model('ecoli')

# Add all metabolites from iJO1366 to the new ME-model
for met in ijo.metabolites:
    me.add_metabolites(met)
for rxn in ijo.reactions:
    me.add_reaction(rxn)

The ME-model contains a "global_info" attribute which stores information used to calculate coupling constraints, along with other functions. The specifics of each of these constraints will be discussed when they are implemented.

*Note*: k_deg will initially be set to 0. We will apply RNA degradation later in the tutorial.

In [3]:
# "Translational capacity" of organism
me.global_info['kt'] =  4.5 #(in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] =  0.087 #scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] =  0. #1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1700. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000. #109. / 1000. # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000. # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000. # in kDA
me.global_info['f_tRNA'] = .12

## Add Metabolic Reaction

The general workflow for adding any reaction to a ME-model using COBRAme occurs in three steps:
1. Create the ProcessData(s) associated with the reaction and populate them with the necessary information
2. Create the MEReaction and link the appropriate ProcessData
3. Execute the MEReaction's update method

-------

MetabolicReactions require, at a minimum, one corresponding StoichiometricData. StoichiometricData essentially holds the information contained in an M-model reaction. This includes the metabolite stoichiometry and the upper and lower bound of the reaction. As a best practice, StoichiometricData typically uses an ID equivalent to the M-model reaction ID.

So first, we will create a StoichiometricData object to define the stoichiometry of the conversion of *a* to *b*

The code below creates a StoichiometricData object which stores all of the stoichiometric information associated with the MetabolicReaction. This includes:
 - Reaction Stoichiometry
 - Lower and upper bound
 
Only one StoichiometricData object should be created for both reversible and irreversible reactions

In [4]:
# unique to COBRAme, construct a stoichiometric data object with the reaction information
data = cobrame.StoichiometricData('a_to_b', me)
stoichiometry = {'a':-1, 'b': 1}
data._stoichiometry = stoichiometry
data.lower_bound = -1000
data.upper_bound = 1000

The StoichiometricData for this reversible reaction is then assigned to two different MetabolicReactions (Due to the enzyme dilution constraint, all enzyme catalyzed reactions must be reverisble; more on this later). The MetabolicReactions require:
 - The associated StoichiometricData
 - The *reverse* flag set to True for reverse reactions, False for forward reactions
 - Enzyme $K_{eff}$ for reaction (discussed later, dafault=65)

These fields are then processed and the actual model reaction is created using the MetabolicReaction's update() function

In [5]:
# Create a forward ME Metabolic Reaction and associate the stoichiometric data to it
rxn = cobrame.MetabolicReaction('a_to_b_FWD_complex_ab')
me.add_reaction(rxn)
rxn.stoichiometric_data = data
rxn.reverse = False
rxn.keff = 65.
# Update
rxn.update(verbose=False)

# Create a reverse ME Metabolic Reaction and associate the stoichiometric data to it
rxn = cobrame.MetabolicReaction('a_to_b_REV_complex_ab')
me.add_reaction(rxn)
rxn.stoichiometric_data = data
rxn.reverse = True
rxn.keff = 65.
# Update
rxn.update(verbose=False)


print me.reactions.a_to_b_FWD_complex_ab.reaction
print me.reactions.a_to_b_REV_complex_ab.reaction

a --> b
b --> a


**Note**: the $k_{eff}$ is not included in the reaction since no complex has been associated to it yet

## Associate a_to_b forward and reverse reaction with a catalyzing enzyme
### Create ComplexData for enzyme
For COBRAme models, the reaction gene-protein-reaction rule (GPR) is replaced with a metabolite representing the synthesis of the enzyme(s) catalyzing a reaction. This metabolite is formed explicitly in a ME model by seperate reaction to transcribe the gene(s) and translate the protein(s) the compose the complex. These reactions will be added later.

ComplexData objects contain:
 - Subunit stoichiometry
 - Modification dictionary (discussed later)
 - Translocation dictionary (discussed later)

In [6]:
data = cobrame.ComplexData('complex_ab', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}

rxn = cobrame.ComplexFormation('formation_complex_ab')
me.add_reaction(rxn)
rxn.complex_data_id = data.id
rxn._complex_id = data.id
rxn.update(verbose=False)
print me.reactions.formation_complex_ab.reaction

protein_a + protein_b --> complex_ab


Alternatively, ComplexData has a ```create_complex_formation()``` function to create the sythesis reaction following the naming conventions. It contains an ```update()``` function which incorporates changes in the ComplexData

In [7]:
data = cobrame.ComplexData('complex_ba', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}
data.create_complex_formation()
print me.reactions.formation_complex_ba.reaction

protein_a + protein_b --> complex_ba


### Apply modification to complex formation reaction
Many enzyme complexes in an ME-model require cofactors or prosthetic groups in order to properly function. Information about such processes are stored as ModificationData.

For instance, we can add the modification of an iron-sulfur cluster, a common prosthetic group, by doing the following:

In [8]:
# Define the stoichiometry of the modification 
mod_data = cobrame.ModificationData('mod_2fe2s_c', me)
mod_data.stoichiometry = {'2fe2s_c': -1}
# this process can also be catalyzed by a chaperone
mod_data.enzyme = 'complex_ba'
mod_data.keff = 65.  # default value

Associate modification to complex and ```update()``` its formation

In [9]:
complex_data = me.complex_data.complex_ab
complex_data.modifications['mod_2fe2s_c'] = 1
print me.reactions.formation_complex_ab.reaction
me.reactions.formation_complex_ab.update()
print me.reactions.formation_complex_ab.reaction

protein_a + protein_b --> complex_ab
2fe2s_c + 4.27350427350427e-6*mu complex_ba + protein_a + protein_b --> 0.17582 biomass + complex_ab


### Associate enzyme with MetabolicReaction

The ComplexData object created in the previous cell can be incorporated into the MetabolicReaction using code below. 

**Note**: the update() function is required to apply the change.

In [10]:
me.reactions.a_to_b_FWD_complex_ab.complex_data = data
print me.reactions.a_to_b_FWD_complex_ab.reaction
me.reactions.a_to_b_FWD_complex_ab.update()
print me.reactions.a_to_b_FWD_complex_ab.reaction, '\n'

me.reactions.a_to_b_REV_complex_ab.complex_data = data
print me.reactions.a_to_b_REV_complex_ab.reaction
me.reactions.a_to_b_REV_complex_ab.update()
print me.reactions.a_to_b_REV_complex_ab.reaction

a --> b
a + 4.27350427350427e-6*mu complex_ba --> b 

b --> a
b + 4.27350427350427e-6*mu complex_ba --> a


The coefficient for complex_ab is determined by the expression $$\frac{\mu}{k_{eff}}$$ which in its entirety represents the dilution of an enzyme following a cell doubling. The coupling constraint can be summarized as followed
$$ 
\begin{align}
&v_{dilution,j} = \mu \sum_{i} \left( \frac{1}{k_{eff,i}} v_{usage,i} \right), & \forall j \in Enzyme
\end{align}
$$

Where $v_{usage,i}$ is the flux through the metabolic reaction, $\mu$ is the growth rate of the cell, and $k_{eff}$ is the turnover rate for the process and conveys the productivity of the enzyme complex. Physically, it can be thought of as the number of reactions the enzyme can catalyze per cell division. 


By default the $k_{eff}$ for a MetabolicReaction is set to 65 but this can be changed using the code below.

In [11]:
me.reactions.a_to_b_FWD_complex_ab.keff = 1000
me.reactions.a_to_b_FWD_complex_ab.update()

# The forward and reverse direction can have differing keffs
print me.reactions.a_to_b_FWD_complex_ab.reaction
print me.reactions.a_to_b_REV_complex_ab.reaction

a + 2.77777777777778e-7*mu complex_ba --> b
b + 4.27350427350427e-6*mu complex_ba --> a


## With building functions

In [12]:
stoich_data = cobrame.StoichiometricData('b_to_c', me)
stoich_data._stoichiometry = {'b': -1, 'c': 1}
stoich_data.lower_bound = 0
stoich_data.upper_bound = 1000.
building.add_metabolic_reaction_to_model(me, stoich_data.id, 'forward', complex_id='complex_ab', update=True)
print me.reactions.b_to_c_FWD_complex_ab.reaction

Created <Metabolite c at 0x7f7436aca650> in <MetabolicReaction b_to_c_FWD_complex_ab at 0x7f743583f4d0>
b + 4.27350427350427e-6*mu complex_ab --> c


## Add transcription and translation reactions


In [12]:
sequence = ("ATG" + "TTT" * 12 + "TAT" * 12 + 
            "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA")
RNA_a = cobrame.TranscribedGene('RNA_a')
me.add_metabolites(RNA_a)
RNA_a.nucleotide_sequence = sequence

### Add the TranslationData for protein_a and protein_b
In order to add a TranslationData object to a ME model the user must additionally specifify the mRNA id and protein id of the translation reaction that will be added. This information as well as a nucleotide sequence is the only information required to add a translation reaction.

In [13]:
data = cobrame.TranslationData('a', me, 'RNA_a', 'protein_a')
data.nucleotide_sequence = sequence

### Add the TranslationReaction for protein_a and protein_b
By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a reaction reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [14]:
rxn = cobrame.TranslationReaction('translation_a')
rxn.translation_data = data
me.add_reaction(rxn)
rxn.update()
print rxn.reaction

0.000498399634202103*mu + 0.000195123456790123 RNA_a + 12 asp__L_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_a + 7.500606 protein_biomass


This reaction also produces a small amount of the a $biomass$ metabolite (constraint). This term has a coefficient corresponding to the molecular weight (in $kDA$) of the protein being translated. This constraint will be implemented into a $v_{biomasss\_dilution}$ reaction with the form: 
$$\mu \leq v_{biomass\_dilution} \leq \mu $$

A mathematical description of the biomass constraint can be found....

The coefficient for RNA_a represents 
$$
\begin{align}
& v_{dilution,j} \geq \frac{3}{\kappa_{\tau} c_{mRNA}} \cdot (\mu + \kappa_{\tau} r_0) v_{translation,j}  , & \forall j \in mRNA \\
\end{align}
$$
where:$\dots$

## Add tRNA charging reaction
The above reaction is not a complete picture of translation considering that it's missing two important features:
1. amino acid addition by tRNAs
2. ribosome

### Add in the tRNAData and tRNAChargingReaction

In [15]:
data = cobrame.tRNAData('tRNA_d_GUA', me, 'val__L_c', 'RNA_d', 'GUA')

In [16]:
rxn = cobrame.tRNAChargingReaction('charging_tRNA_d_GUA')
me.add_reaction(rxn)
rxn.tRNAData = data

#Setting verbose=False suppresses print statements indicating that new metabolites were created
rxn.update(verbose=False)
print rxn.reaction

0.000116266666666667*mu + 4.55184e-5 RNA_d + 0.000116266666666667*mu + 4.55184e-5 val__L_c --> generic_tRNA_GUA_val__L_c


This reaction creates one "generic_charged_tRNA" equivalement that can then be used in a translation reaction

The coefficient for ```RNA_d``` and ```lys__L_c``` are defined by:

$$
\begin{align}
v_{dilution,j} \geq \frac{1}{\kappa_{\tau} c_{tRNA,j}} (\mu + \kappa_{\tau} r_0)  v_{charging,j} , & \forall j \in tRNA
\end{align}
$$

In [17]:
data.synthetase = 'synthetase'
rxn.update(verbose=False)
print rxn.reaction

0.000116266666666667*mu + 4.55184e-5 RNA_d + 4.27350427350427e-6*mu*(0.000116266666666667*mu + 1.0000455184) synthetase + 0.000116266666666667*mu + 4.55184e-5 val__L_c --> generic_tRNA_GUA_val__L_c


A more complete mathematical description of the tRNA synthetase coupling constraints can be found in the 'tRNA_mod.ipynb'

### Add tRNAs to translation

Here we take advantage of an additional subclass of ProcessData, called a SubreactionData object. This class is used to lump together processeses that occur as a result of many individual reactions, including translation elongation, ribosome formation, tRNA charging, etc. Since each of these steps often involve an enzyme that requires its own dilution coupling, this process allows these processes to be lumped into one reaction while still enabling each subprocess to be modified.

Below, we add the SubreactionData (excluding enzymes) for the addition of each amino acid using information from the E. coli codon table

In [18]:
data = cobrame.SubreactionData('val_addition_at_GUA', me)
data.stoichiometry = {'generic_tRNA_GUA_val__L_c': -1,
                      'gtp_c': -1, 'gdp_c': 1, 'h_c': 1,
                      'pi_c': 1}
data.enzyme = 'elongation_factor'

In [19]:
me.translation_data.a.subreactions['val_addition_at_GUA'] = 5.
print me.reactions.translation_a.reaction, '\n'
me.reactions.translation_a.update(verbose=False)
print me.reactions.translation_a.reaction

0.000498399634202103*mu + 0.000195123456790123 RNA_a + 12 asp__L_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_a + 7.500606 protein_biomass 

0.000498399634202103*mu + 0.000195123456790123 RNA_a + 12 asp__L_c + 2.13675213675214e-5*mu elongation_factor + 5.0 generic_tRNA_GUA_val__L_c + 5.0 gtp_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 5.0 gdp_c + 5.00000000000000 h_c + 5.00000000000000 pi_c + protein_a + 7.500606 protein_biomass


## Add transcription reaction (incomplete)

In [20]:
data = cobrame.TranscriptionData('TU_a',me,RNA_products={'RNA_a'})
data.nucleotide_sequence = sequence

In [21]:
rxn = cobrame.TranscriptionReaction('transcription_TU_a')
rxn.transcription_data = data
me.add_reaction(rxn)
rxn.update()
print rxn.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_a + 186 h_c + 186 ppi_c


In [22]:
# RNA type required to add the biomass constraint
me.metabolites.RNA_a.RNA_type = 'mRNA'
rxn.update()
print rxn.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_a + 186 h_c + 59.172286 mRNA_biomass + 186 ppi_c


In [23]:
from cobrame.solve.algorithms import binary_search
for m in ['a', 'b']:
    r = cobrame.Reaction('source_'+ m)
    me.add_reaction(r)
    stoich = -1 if m == 'a' else 1
    r.add_metabolites({m: stoich})
    
binary_search(me, mu_accuracy=1e-2)

LP files will be saved in /tmp/tmpElx1mu
mu   	status	reset	time	iter	obj
0.000	+	False	0.52	3619	0.98237751966
2.000	-	False	0.03	78	
1.000	-	False	0.01	18	
0.500	-	False	0.01	20	
0.250	-	False	0.01	20	
0.125	-	False	0.02	19	
0.062	-	False	0.01	10	
0.031	-	False	0.01	10	
0.016	-	False	0.01	10	
0.008	-	False	0.01	10	
0.000	+	False	0.01	0	0.98237751966
completed in 1.3 seconds and 11 iterations


<Solution 0.00 at 0x7fc0005ddb90>

In [36]:
me.reactions.source_b.upper_bound

1000.0

## Alternatively, using COBRAme utility functions (incomplete)

COBRAme includes a set of utility functions contained in *building* that simplifies model buidling and imposes the best practices that we suggest.

In [25]:
from cobrame.util import building

In [26]:

building.create_transcribed_gene(me, 'c', 1, 2, sequence, '+', 'mRNA')

<TranscribedGene RNA_c at 0x7fbfffd69390>

In [27]:
building.add_transcription_reaction(me, 'TU_c', {'c'}, sequence)

<TranscriptionReaction transcription_TU_c at 0x7fbfffd69490>

In [28]:
print me.reactions.transcription_TU_c.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_c + 186 h_c + 59.172286 mRNA_biomass + 186 ppi_c


In [29]:
building.add_translation_reaction(me, 'c', dna_sequence=sequence, update=True)
print me.reactions.translation_c.reaction

0.000498399634202103*mu + 0.000195123456790123 RNA_c + 12 asp__L_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 7.500606 protein_biomass + protein_c
